In [0]:
secret=dbutils.secrets.get('newscope12','scope')

In [0]:
client_id = "88dec17f-62f0-4cab-82b7-b9022c2cc21c"
client_secret = secret
directory_id="037e0179-cd0e-4201-951c-167b7554d77a"

spark.conf.set("fs.azure.account.auth.type.storageforcustomers.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.storageforcustomers.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.storageforcustomers.dfs.core.windows.net", client_id)
spark.conf.set("fs.azure.account.oauth2.client.secret.storageforcustomers.dfs.core.windows.net", client_secret)
spark.conf.set("fs.azure.account.oauth2.client.endpoint.storageforcustomers.dfs.core.windows.net", f"https://login.microsoftonline.com/{directory_id}/oauth2/token")

In [0]:
display(dbutils.secrets.listScopes())
display(dbutils.secrets.list('newscope12'))

name
newscope
newscope12
training_scope


key
scope


In [0]:
display(dbutils.fs.ls("abfss://raw@storageforcustomers.dfs.core.windows.net"))

path,name,size,modificationTime
abfss://raw@storageforcustomers.dfs.core.windows.net/accounts.csv,accounts.csv,2331,1730905206000
abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/,cleaned_data/,0,1730934907000
abfss://raw@storageforcustomers.dfs.core.windows.net/customers.csv,customers.csv,4603,1730905207000
abfss://raw@storageforcustomers.dfs.core.windows.net/loan_payments.csv,loan_payments.csv,2613,1730905205000
abfss://raw@storageforcustomers.dfs.core.windows.net/loans.csv,loans.csv,2340,1730905209000
abfss://raw@storageforcustomers.dfs.core.windows.net/transactions.csv,transactions.csv,3513,1730905207000


In [0]:
df_accounts = spark.read.format("delta").load("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/accounts.csv")
df_customers = spark.read.format("delta").load("abfss://raw@storageforcustomers.dfs.core.windows.net/cleaned_data/customers.csv")
df_accounts.show(20)
df_customers.show(20)

+----------+-----------+------------+-------+
|account_id|customer_id|account_type|balance|
+----------+-----------+------------+-------+
|         1|         45|     Savings| 1000.5|
|         2|         12|    Checking|2500.75|
|         3|         78|     Savings| 1500.0|
|         4|         34|    Checking|3000.25|
|         5|         56|     Savings|  500.0|
|         6|         23|    Checking| 1200.5|
|         7|         89|     Savings| 800.75|
|         8|         67|    Checking| 2200.0|
|         9|         14|     Savings| 900.25|
|        10|         92|    Checking| 1800.5|
|        11|          3|     Savings|1100.75|
|        12|         81|    Checking| 2700.0|
|        13|         29|     Savings|1300.25|
|        14|         64|    Checking| 3200.5|
|        15|         47|     Savings| 700.75|
|        16|         18|    Checking| 1400.0|
|        17|         99|     Savings| 600.25|
|        18|          5|    Checking| 1600.5|
|        19|         76|     Savin

In [0]:
from pyspark.sql.functions import *

# Perform the transformation: calculate total balance for each customer
transformed_df = df_customers.join(df_accounts, "customer_id", "left") \
    .groupBy(*df_customers.columns) \
    .agg(
        sum("balance").alias("total_balance"),
        *[first(col).alias(col) for col in df_accounts.columns if col != "customer_id"]
    ) \
    .orderBy(asc("customer_id"))

transformed_df.show(50)


+-----------+-----------+----------+----------------+--------------+-----+------+-------------+----------+------------+-------+
|customer_id| first_name| last_name|         address|          city|state|   zip|total_balance|account_id|account_type|balance|
+-----------+-----------+----------+----------------+--------------+-----+------+-------------+----------+------------+-------+
|          1|       John|       Doe|      123 Elm St|       Toronto|   ON|M4B1B3|       8900.0|        88|    Checking| 8900.0|
|          2|       Jane|     Smith|   456 Maple Ave|        Ottawa|   ON|K1A0B1|       8300.5|        82|    Checking| 8300.5|
|          3|    Michael|   Johnson|      789 Oak Dr|      Montreal|   QC|H1A1A1|      1100.75|        11|     Savings|1100.75|
|          4|      Emily|     Davis|     101 Pine Rd|       Calgary|   AB|T2A0A1|       7900.5|        78|    Checking| 7900.5|
|          5|      David|    Wilson|  202 Birch Blvd|     Vancouver|   BC|V5K0A1|       1600.5|        1

In [0]:
transformed_df.write.format("delta").mode("overwrite").save("abfss://gold@storageforcustomers.dfs.core.windows.net/customer_total_balances")

In [0]:
transformed_df.show(50)

+-----------+-----------+----------+----------------+--------------+-----+------+-------------+----------+------------+-------+
|customer_id| first_name| last_name|         address|          city|state|   zip|total_balance|account_id|account_type|balance|
+-----------+-----------+----------+----------------+--------------+-----+------+-------------+----------+------------+-------+
|          1|       John|       Doe|      123 Elm St|       Toronto|   ON|M4B1B3|       8900.0|        88|    Checking| 8900.0|
|          2|       Jane|     Smith|   456 Maple Ave|        Ottawa|   ON|K1A0B1|       8300.5|        82|    Checking| 8300.5|
|          3|    Michael|   Johnson|      789 Oak Dr|      Montreal|   QC|H1A1A1|      1100.75|        11|     Savings|1100.75|
|          4|      Emily|     Davis|     101 Pine Rd|       Calgary|   AB|T2A0A1|       7900.5|        78|    Checking| 7900.5|
|          5|      David|    Wilson|  202 Birch Blvd|     Vancouver|   BC|V5K0A1|       1600.5|        1